In [7]:
import pandas as pd
import numpy as np
from random import sample
from itertools import combinations
from scipy.io import loadmat, savemat
from sklearn.cluster import SpectralClustering, KMeans
from typing import Sequence, List, Set
from clustering import spectral_partition
import random
import math


In [8]:
def clustering_to_labeling(clusters):
    labels = np.zeros(sum(map(len, clusters)), dtype=int)
    for k, cluster in enumerate(clusters):
        for v in cluster:
            labels[v] = k
    return labels


In [9]:
def vi(predicted_labels: Sequence[int],
       true_labels: Sequence[int]) -> float:
    def expand(labels: Sequence[int]) -> List[Set[int]]:
        expanded = {label: set() for label in labels}
        for node, label in enumerate(labels):
            expanded[label].add(node)
        return list(expanded.values())

    n = len(predicted_labels)
    predicted = expand(predicted_labels)
    p = list(map(lambda x: len(x) / n, predicted))
    true = expand(true_labels)
    q = list(map(lambda x: len(x) / n, true))
    r = [[len(predicted[i].intersection(true[j])) / n
          for j in range(len(true))]
         for i in range(len(predicted))]
    vi = sum([r[i][j] * (np.log(r[i][j] / p[i]) + np.log(r[i][j] / q[j]))
              if r[i][j] > 0 else 0
              for j in range(len(true))
              for i in range(len(predicted))]) * -1
    return abs(vi)


In [10]:
def label_to_cluster(labels):
    num_clusters = len(set(labels))
    v_hat = [set() for _ in range(num_clusters)]
    for i, label in enumerate(labels):
        v_hat[label].add(i)
    return v_hat

In [11]:
from itertools import product


def edge_error_rate(predicted_v, true_v):
    n = sum(map(len, true_v))
    A_hat = np.zeros((n, n), dtype=int)
    A = np.zeros((n, n), dtype=int)
    for cluster in map(list, predicted_v):
        for i, j in product(cluster, cluster):
            A_hat[i, j] = 1
    for cluster in map(list, true_v):
        for i, j in product(cluster, cluster):
            A[i, j] = 1
    return (A_hat != A).sum() / (n * n)


In [12]:
T = 813210
K = 90
pair_prob_mat = loadmat(f'../../mat/pair_prob_mat_K={K}.mat')['P']
labels_true = sum([[_k] * int(900 / K) for _k in range(K)], [])

In [13]:
from random import choices
random.seed(0)
np.random.seed(0)

yun_vi_s = []
yun_err_s = []

for it in range(10):
    A = np.zeros((900, 900), dtype=int)
    for i, j in choices(list(product(range(900), range(900))), k=T):
        A[i, j] += 1 if np.random.uniform() < pair_prob_mat[i, j] else 0

    cluster_yun = spectral_partition(A)
    labels_yun = clustering_to_labeling(cluster_yun)
    voi_yun = vi(labels_yun, labels_true)
    edge_err_yun = edge_error_rate(cluster_yun, label_to_cluster(labels_true))

    yun_vi_s.append(voi_yun)
    yun_err_s.append(edge_err_yun)